# Data Analysis of a JSON output from the Unity Experiment Application

Given JSON data, a variety of analytical techniques will be applied in order to interpret the given data

## Rotor Derivations

In [11]:
from sympy import symbols
from galgebra.ga import Ga
from galgebra.printer import Format

Format(Fmode = False, Dmode = True)
s4coords = (x,y,z,w) = symbols('x y z w', real=True)
s4 = Ga('e',
g=[1,1,1,1],
coords=s4coords)

### Equations used in Rotor.py

In [8]:
# Rotation of a vector using a Rotor

# Vector
a = s4.mv('a','vector')

# Rotor
b = s4.mv('b','bivector')
s = s4.mv('s', 'scalar')
p = s4.mv('p', 'pseudo')
rotor = b + s + p

(rotor * a * rotor.rev()).Fmt(3)

 (2*a__w*b__xw*s + 2*a__w*b__xy*b__yw + 2*a__w*b__xz*b__zw + 2*a__w*b__yz*p__xyzw - a__x*b__xw**2 - a__x*b__xy**2 - a__x*b__xz**2 + a__x*b__yw**2 + a__x*b__yz**2 + a__x*b__zw**2 - a__x*p__xyzw**2 + a__x*s**2 - 2*a__y*b__xw*b__yw + 2*a__y*b__xy*s - 2*a__y*b__xz*b__yz + 2*a__y*b__zw*p__xyzw - 2*a__z*b__xw*b__zw + 2*a__z*b__xy*b__yz + 2*a__z*b__xz*s - 2*a__z*b__yw*p__xyzw)*e_x
 + (-2*a__w*b__xw*b__xy - 2*a__w*b__xz*p__xyzw + 2*a__w*b__yw*s + 2*a__w*b__yz*b__zw - 2*a__x*b__xw*b__yw - 2*a__x*b__xy*s - 2*a__x*b__xz*b__yz - 2*a__x*b__zw*p__xyzw + a__y*b__xw**2 - a__y*b__xy**2 + a__y*b__xz**2 - a__y*b__yw**2 - a__y*b__yz**2 + a__y*b__zw**2 - a__y*p__xyzw**2 + a__y*s**2 + 2*a__z*b__xw*p__xyzw - 2*a__z*b__xy*b__xz - 2*a__z*b__yw*b__zw + 2*a__z*b__yz*s)*e_y
 + (-2*a__w*b__xw*b__xz + 2*a__w*b__xy*p__xyzw - 2*a__w*b__yw*b__yz + 2*a__w*b__zw*s - 2*a__x*b__xw*b__zw + 2*a__x*b__xy*b__yz - 2*a__x*b__xz*s + 2*a__x*b__yw*p__xyzw - 2*a__y*b__xw*p__xyzw - 2*a__y*b__xy*b__xz - 2*a__y*b__yw*b__zw - 2*a__y*b_

In [9]:
# "Adding" 2 rotors by Multiplying 2 Rotors

a_b = s4.mv('a_b','bivector')
a_s = s4.mv('a_s', 'scalar')
a_p = s4.mv('a_p', 'pseudo')
rotor_a = a_b + a_s + a_p

b_b = s4.mv('b_b','bivector')
b_s = s4.mv('b_s', 'scalar')
b_p = s4.mv('b_p', 'pseudo')
rotor_b = b_b + b_s + b_p

(rotor_a * rotor_b).Fmt(3)

 -a_b__xw*b_b__xw - a_b__xy*b_b__xy - a_b__xz*b_b__xz - a_b__yw*b_b__yw - a_b__yz*b_b__yz - a_b__zw*b_b__zw + a_p__xyzw*b_p__xyzw + a_s*b_s
 + (-a_b__xw*b_b__yw + a_b__xy*b_s - a_b__xz*b_b__yz + a_b__yw*b_b__xw + a_b__yz*b_b__xz - a_b__zw*b_p__xyzw - a_p__xyzw*b_b__zw + a_s*b_b__xy)*e_x^e_y
 + (-a_b__xw*b_b__zw + a_b__xy*b_b__yz + a_b__xz*b_s + a_b__yw*b_p__xyzw - a_b__yz*b_b__xy + a_b__zw*b_b__xw + a_p__xyzw*b_b__yw + a_s*b_b__xz)*e_x^e_z
 + (a_b__xw*b_s + a_b__xy*b_b__yw + a_b__xz*b_b__zw - a_b__yw*b_b__xy - a_b__yz*b_p__xyzw - a_b__zw*b_b__xz - a_p__xyzw*b_b__yz + a_s*b_b__xw)*e_x^e_w
 + (-a_b__xw*b_p__xyzw - a_b__xy*b_b__xz + a_b__xz*b_b__xy - a_b__yw*b_b__zw + a_b__yz*b_s + a_b__zw*b_b__yw - a_p__xyzw*b_b__xw + a_s*b_b__yz)*e_y^e_z
 + (a_b__xw*b_b__xy - a_b__xy*b_b__xw + a_b__xz*b_p__xyzw + a_b__yw*b_s + a_b__yz*b_b__zw - a_b__zw*b_b__yz + a_p__xyzw*b_b__xz + a_s*b_b__yw)*e_y^e_w
 + (a_b__xw*b_b__xz - a_b__xy*b_p__xyzw - a_b__xz*b_b__xw + a_b__yw*b_b__yz - a_b__yz*b_b__yw + a_b__z

## Get JSON Data

In [12]:
import Rotor
import json
import math
import numpy as np

# user <- 10 or test
user = "test"
filename = str(user) + ".json"
data = ""

print("Reading \"" + filename + "\"")

try:
    with open(filename, "r") as f:
        data = json.load(f)
    print("Read json data into [data]")
    print(json.dumps(data, indent=4))
except Exception as e:
    print(e)


Reading "test.json"
Read json data into [data]
{
    "Control": {
        "Shape_Match": {
            "Shape_Match0": {
                "Loaded Shape": "TorusR123",
                "Selected Shape": "Torus",
                "Texture": 0,
                "Time": 2.09690093994141,
                "Initial Rotation": [
                    -0.1314062,
                    0.2928154,
                    -0.4285405,
                    -0.267429,
                    0.2637328,
                    0.5081967,
                    -0.5503097,
                    0.1056734
                ],
                "Final Rotation": [
                    -0.1314062,
                    0.2928154,
                    -0.4285405,
                    -0.267429,
                    0.2637328,
                    0.5081967,
                    -0.5503097,
                    0.1056734
                ],
                "W Count": 0,
                "Swipe Count": 2
            },
            "Shape_Match0_Sur

In [13]:
# Representation Order
order = [i for i in data.keys()]
print(order)

['Control', '4D-3D', 'Timeline', 'Multi-View']


### Feature Engineering

In [14]:
# Remove low confidence and low submission time (Accidentley clicked submit?)


### Analysis of Shape Matching

In [15]:
test_count = 5

# The number of correct answers per representation
def correct_per_rep(data):
    count = {}
    # iterate through each representation
    # assign its correct answer count to 0
    for rep, v in data.items():
        count[rep] = 0

        # for each test, if the answer was correct 
        # add it to the correct answer count
        for i in range(test_count):

            # Loaded shape may have extended parameters (e.g ConeW)
            # But respondent only has option "Cone"
            if v["Shape_Match"]["Shape_Match{0}".format(i)]["Selected Shape"] in \
               v["Shape_Match"]["Shape_Match{0}".format(i)]["Loaded Shape"]:
                count[rep] += 1
    return count

# The average time per representation
def average_time_per_rep(data):
    average_time = {}
    # iterate through each representation
    # initialise average time per representation
    for rep, v in data.items():
        sum = 0

        # sum up the time for the representation
        for i in range(test_count):
            sum += v["Shape_Match"]["Shape_Match{0}".format(i)]["Time"]

        average_time[rep] = sum/test_count
    return average_time

# Use of the W axis slider per representation
def use_W_per_rep(data):
    count = {}
    # iterate through each representation
    # assign its w axis use count to 0
    for rep, v in data.items():
        count[rep] = 0

        # for each test, if the W axis slider was used, increment count
        for i in range(test_count):
            count[rep] += v["Shape_Match"]["Shape_Match{0}".format(i)]["W Count"]
    return count

# Use of rotating the object, per representation
def use_of_rotation_per_rep(data):
    count = {}
    # iterate through each representation
    # assign its use of rotation count to 0
    for rep, v in data.items():
        count[rep] = 0

        for i in range(test_count):
            count[rep] += v["Shape_Match"]["Shape_Match{0}".format(i)]["Swipe Count"]
    return count


# Take the initial rotor and the final rotor and find the difference between them
# if there is a difference, note that the rotation was used
"""
a = v["Shape_Match"]["Shape_Match{0}".format(i)]["Initial Rotation"]
ra = Rotor.Rotor4()
ra.constructor(a[0], a[1], a[2], a[3], a[4], a[5], a[6], a[7])

b = v["Shape_Match"]["Shape_Match{0}".format(i)]["Final Rotation"]
rb = Rotor.Rotor4()
rb.constructor(b[0], b[1], b[2], b[3], b[4], b[5], b[6], b[7])

if (Rotor.Rotor4.difference(ra, rb) > 0.00001):
"""

'\na = v["Shape_Match"]["Shape_Match{0}".format(i)]["Initial Rotation"]\nra = Rotor.Rotor4()\nra.constructor(a[0], a[1], a[2], a[3], a[4], a[5], a[6], a[7])\n\nb = v["Shape_Match"]["Shape_Match{0}".format(i)]["Final Rotation"]\nrb = Rotor.Rotor4()\nrb.constructor(b[0], b[1], b[2], b[3], b[4], b[5], b[6], b[7])\n\nif (Rotor.Rotor4.difference(ra, rb) > 0.00001):\n'

In [16]:
#  Call Functions

# The number of correct answers per representation
print(correct_per_rep(data))
# was there continuous improvement with each new representation
#   (Overlay the graphs)

# The average time per representation
print(average_time_per_rep(data))
# The average time for correct answers
# The average time for incorrect answers

# Which representations provided more information about about the shape of the object in terms of the W axis
#   (order representations by the fact a user did not move the W axis slider at all)
print(use_W_per_rep(data))
# Which representations provided more information about about the shape from all sides?
#   (order representations based on the fact the initial random rotation is equal to the final random rotation)
print(use_of_rotation_per_rep(data))

# Correlations
# Is there a correlation between "Loaded Shape" and "Can you relate this to a 2D Cross Section of a 3D Shape" Survey Question
# Is there a correlation between incorrect answers and minimum angle of the initial rotation of the object 
#   (Range and mean of the initial rotation)
#   Is there a correlation between incorrect answers and an un-rotated object - capsule and cone
# Is there a correlation between incorrect answers and "Loaded Shape"
# Is there a correlation between confidence and "Loaded Shape"
# Is there a correlation between confidence and time taken
# Is there a correlation between confidence and representation

{'Control': 5, '4D-3D': 5, 'Timeline': 5, 'Multi-View': 5}
{'Control': 3.923245239257814, '4D-3D': 3.42012939453125, 'Timeline': 5.770849609375, 'Multi-View': 2.5972412109375}
{'Control': 2, '4D-3D': 1, 'Timeline': 1, 'Multi-View': 0}
{'Control': 20, '4D-3D': 15, 'Timeline': 16, 'Multi-View': 12}


### Analysis of Rotation Matching

In [ ]:
# Call Functions

# What is correct here?

# was there continuous improvement with each new representation
#   (Overlay the graphs)
# use number of planes of rotation to weight effectiveness of representations

# The average time per representation
# The average time for correct answers
# The average time for incorrect answers

# Correlations
# Is there a correlation between "Loaded Shape" and "Can you relate this to a 2D Cross Section of a 3D Shape" Survey Question
# Is there a correlation between correctness and "Loaded Shape"
# Is there a correlation between correctness and "Loaded Texture"
# Is there a correlation between confidence and "Loaded Shape"
# Is there a correlation between confidence and time taken
# Is there a correlation between confidence and representation
# Is there a correlation between confidence and correctness
# Is there a correlation between incorrect answers and 3D rotation
#   (Does it confuse things even more when combined with 4D rotation)

### Analysis of Pose Matching

In [69]:
# Angle in Degrees
def rad_deg(x):
    return (x/(2*math.pi) ) *360

In [ ]:
# Call Functions

# The average "Accuracy" per representation
# was there continuous improvement with each new representation
#   (Overlay the graphs)

# The average Time per representation

# Correlation
# Is there a correlation between Time taken and "How easy did you find it"
# Is there a correlation between Time taken and "Accuracy"
#   (Assuming No, unless ran out of time?)
# Is there a correlation between "Accuracy" and "How easy did you find it"
# Is there a correlation between "Loaded Shape" and "Accuracy"
# Is there a correlation between "Loaded Texture" and "Accuracy"